## Setting the pre-requirements

In [1]:
!pip install tensorflow_gpu==1.15

     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 3.8MB 53.0MB/s 
     |████████████████████████████████| 512kB 53.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=b18acf138244afa365cd11309c966d460d2a4b8e9195ef08d61ea696cf3a1573
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [3]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 5000  # 200000 to improve

# Number of evaluation steps.
#num_eval_steps = 50

MODELS_CONFIG = {
        'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'demo.config',
        'batch_size': 1
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [4]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

!pip install tf_slim


%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content
Cloning into 'tensorflow-object-detection-faster-rcnn'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 24.84 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (412/412), done.
/content/tensorflow-object-detection-faster-rcnn
Already up to date.
     |████████████████████████████████| 358kB 2.8MB/s 
/content
Selecting previously unselected package python-bs4.
(Reading database ... 144600 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package p

In [6]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

!echo {DEST_DIR}
!ls -alh {DEST_DIR}

fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)
print(pipeline_fname)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

/content/models/research
/content/models/research/pretrained_model
total 111M
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 .
drwxr-xr-x 24 root   root 4.0K Sep 17 04:31 ..
-rw-r--r--  1 345018 5000   77 Feb  1  2018 checkpoint
-rw-r--r--  1 345018 5000  55M Feb  1  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 5000  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 5000  16K Feb  1  2018 model.ckpt.index
-rw-r--r--  1 345018 5000 5.5M Feb  1  2018 model.ckpt.meta
-rw-r--r--  1 345018 5000 3.2K Feb  1  2018 pipeline.config
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 saved_model
/content/models/research/object_detection/samples/configs/demo.config


## Create different sets of train and eval datasets

In [7]:
%cd /content/

# change the name when re-running
# name of the file where the data will go when we do splitting
split_output_file_name = "Splited_record"
!mkdir {split_output_file_name}

# Change the output file name
out_put_file_name = "Records"

# Number of splits
num_of_splits = 6

# Main tfrecord file name
file_path = "/content/tensorflow-object-detection-faster-rcnn/Original_dataset/eval_default.tfrecord"

# # dir where the split datasets will be stored
# output_dir_name = split_output_file_name

/content


In [11]:
%cd ..

/content


In [12]:
import os
import tensorflow as tf

# sum of records
def give_sum_of_records(file_path):
  print("File name: ",file_path.split("/")[-1],", Number of records: ",sum(1 for _ in tf.data.TFRecordDataset(file_path)))

# Split the record file
def split_the_record(file_path,num_of_splits,output_dir_name):
  %cd {output_dir_name}
  raw_dataset = tf.data.TFRecordDataset(file_path)
  shards = num_of_splits
  for i in range(shards):
      writer = tf.data.experimental.TFRecordWriter(f"file_{i}.tfrecord")
      print(writer)
      writer.write(raw_dataset.shard(shards, i))

split_the_record(file_path,num_of_splits,split_output_file_name)

/content/Splited_record


In [ ]:
# listing the files created

file_path = "/content/"+output_dir_name+"/"
list_of_records = os.listdir(file_path)
list_of_records.sort()
list_of_records = [file_path+x for x in list_of_records]

for i in range(len(list_of_records)):
  print(list_of_records[i])

print("\nNumber of records:",len(list_of_records))

In [ ]:

!mkdir /content/{out_put_file_name}

%cd /content/{out_put_file_name}

for i in range(0,len(list_of_records)):
  print("Eval:",list_of_records[i])
  temp = [list_of_records[x] for x in range(len(list_of_records)) if x != i]
  print("train:",temp,"\n")

  # For eval dataset
  list_of_tfrecord_files = list_of_records[i]
  dataset = tf.data.TFRecordDataset(list_of_tfrecord_files)

  # # Save dataset to .tfrecord file
  filename = 'eval_'+str(i)+'.tfrecord'

  writer = tf.data.experimental.TFRecordWriter(filename)
  writer.write(dataset)

  # # For training dataset
  list_of_tfrecord_files = temp
  dataset = tf.data.TFRecordDataset(list_of_tfrecord_files)

  # # Save dataset to .tfrecord file
  filename = 'train_'+str(i)+'.tfrecord'

  writer = tf.data.experimental.TFRecordWriter(filename)
  writer.write(dataset)

In [ ]:
file_path = "/content/"+out_put_file_name+"/"

list_of_records = os.listdir(file_path)
list_of_records.sort()
list_of_records = [file_path+x for x in list_of_records]

for record in list_of_records:
  give_sum_of_records(record)

print("\nNumber of records:",len(list_of_records))

## Now for training

In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
train_record_fname = '/content/tensorflow-object-detection-faster-rcnn/data/data.record'
label_map_pbtxt_fname = '/content/tensorflow-object-detection-faster-rcnn/data/demo.pbtxt'

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
import re

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    
    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

!cat {pipeline_fname}